In [10]:
import numpy as np
import pyniggli
from opf_python.lat_type import lat_type
from opf_python.niggli_lat_id import niggli_id
from opf_python.test_basis import test_cell

In [18]:
def niggli_setting(A):
    """Finds the corrdinates in G6 for the lattice A. Where G6 is
    [A[:,0].A[:,0],A[:,1].A[:,1],A[:,2].A[:,2],
    2*A[:,1].A[:,2],2*A[:,2].A[:,0],2*A[:,0].A[:,1]]
    
    Args:
        A (numpy array): The lattice vectors as columns of a matrix.
        
    Returns:
        G (list): The coordinates on G6 space.
    """
    
    a = A[:,0]
    b = A[:,1]
    c = A[:,2]
    A = np.dot(a,a)
    B = np.dot(b,b)
    C = np.dot(c,c)
    xi = np.dot(b,c)
    eta = np.dot(a,c)
    zeta = np.dot(a,b)
    
    G = [A,B,C,xi,eta,zeta]
    return G

# The Idea

The idea here is to try and see if we can figure out a method to transform the symmetry preserving supercell from our basis to the users basis by using the niggli reduced cell. The workflow would be as follows:

1) identify which of the 44 niggli cells is in use (there are two triclinic cells included in those 44 which we'll disregard)

2) Find the Supercell in the niggli basis

3) Transform back to the users basis.

UC = N

S = NHCinv

# Face centered cubic

## First basis

Face centered cubic makes a good test case for the cubic cells, all cubic cells have a single unique niggli cell.

In [4]:
U = np.transpose([[0.5,0,0.5],[0.5,0.5,0],[0,0,1]])
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

Using the brute force method I've now confirmed that this niggli cell has the same symmetry preserving HNFs as our original basis choice.

In [5]:
test_cell(U)

('Users niggli number: ', 1)


True

## Second FCC basis

Let's do a second basis just to be sure.

In [6]:
U = np.transpose([[1,1,2],[1,0,1],[2,1,1]])

In [7]:
test_cell(U)

('Users niggli number: ', 1)


True

## FCC rotated basis (10 degrees about the z-axis)

In [8]:
R = [[np.cos(10.*np.pi/180.),-np.sin(10.*np.pi/180.),0],
     [np.sin(10.*np.pi/180.),np.cos(10.*np.pi/180.),0],
    [0,0,1]]
U = np.dot(R,U)

In [9]:
test_cell(U)

('Users niggli number: ', 1)


True

# Simple Cubic

In [10]:
As = [[[1,0,0],[0,1,0],[0,0,1]], [[1,0,0],[1,1,0],[0,0,1]],
        [[1,0,1],[1,1,0],[0,0,1]],[[1,1,1],[0,1,1],[1,0,1]],[[1,0,2],[0,1,0],[0,1,1]]]
As = [np.transpose(i) for i in As]

In [11]:
for U in As:
    print(U,test_cell(U))

('Users niggli number: ', 3)
(array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 1, 0],
       [0, 1, 0],
       [1, 0, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]]), True)
('Users niggli number: ', 3)
(array([[1, 0, 0],
       [0, 1, 1],
       [2, 0, 1]]), True)


# Body Centered Cubic

In [12]:
As = [[[-1,1,1],[1,-1,1],[1,1,-1]],[[1,0,0],[0,1,0],[0.5,0.5,0.5]],
        [[1,0,0],[0.5,0.5,0.5],[0,0,1]],[[0.5,-0.5,0.5],[0,1,0],[0,0,1]],
        [[0,0,2],[1,-1,1],[1,1,-1]],[[-1,1,1],[2,0,0],[0,2,0]],
        [[1,-1,3],[1,-1,1],[1,1,-1]]]
As = [np.transpose(i) for i in As]

In [14]:
for U in As:
    print(test_cell(U))

('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True
('Users niggli number: ', 5)
True


# Simple Tetragonal

Now we should check a second crystal choice. Simple tetragonal cell's are another where we already have the symmetry preserving HNFs for one of the two niggli cells. We'll have to restrict ourselves to the niggli setting with lattice norms of [A,A,C] where C > A for these tests. The following 4 lattices satisfy that condition.

In [16]:
a1 = np.array([1,0,0])
a2 = np.array([0,1,0])
a3 = np.array([0,0,2])

As = [[a1+a2,a2,a3+a1],[a1+a2+a3,a2+a3,a1+a2],[a1,a2+a1,a3+a3],[-a1,-a2+a3,a3]]
As = [np.transpose(i) for i in As]

In [17]:
for U in As:
    print(test_cell(U))

('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True
('Users niggli number: ', 11)
True


## Third Basis Choice (rotated by 15 degrees)

In [18]:
R = [[np.cos(15.*np.pi/180.),-np.sin(15.*np.pi/180.),0],
     [np.sin(15.*np.pi/180.),np.cos(15.*np.pi/180.),0],
    [0,0,1]]
U = np.random.random()*np.transpose(As[2])
U = np.dot(R,U)

In [19]:
test_cell(U)

('Users niggli number: ', 11)


True

## Fourth Basis Choice (rotated by 27 degrees)

In [23]:
R = [[np.cos(27.*np.pi/180.),-np.sin(27.*np.pi/180.),0],
     [np.sin(27.*np.pi/180.),np.cos(27.*np.pi/180.),0],
    [0,0,1]]
U = np.random.random()*np.transpose(As[3])
U = np.dot(R,U)

In [24]:
test_cell(U)

('Users niggli number: ', 11)


True

# Simple Orthorhombic

In [3]:
As = [[[1.00000000, 0.00000000, 0.00000000],
       [0.00000000, 1.73205080, 0.00000000],
       [0.00000000, 0.00000000, 3.26598640]],
      [[0.50000000, 0.50000000, 0.00000000],
      [0.00000000, 0.00000000, 1.00000000],
      [1.50000000, -1.50000000, 0.00000000]],
      [[0.50000000, 0.50000000, 0.00000000],
      [0.00000000, 0.00000000, 1.00000000],
      [2.00000000, -2.00000000, 0.00000000]]]
As = [np.transpose(i) for i in As]

In [4]:
for U in As:
    print(test_cell(U,eps=1E-8))

('Users niggli number: ', 32)
True
('Users niggli number: ', 32)
True
('Users niggli number: ', 32)
True


# Conclusion

The tests so far indicate that the transformation to the Niggli cell will work. However there are still 39 cases to check and verify in addition to a more detailed study even of these systems.

What I need to do is find a way to quickly identify the lattice type, this can be accomplished via niggli faster than the current method, then verify that the supercell's retain symmetry for a larger test case.

In other words I need a far more extensive testing suite that gets wrapped into a subroutine so that it will automatically generate supercells for multiple densities for a given cell and for each supercell verify that the cell has equal or greater symmetry than the parent. If ever this isn't the case then we'll need to have it stop and report the error, i.e., let us know that the method doesn't actually work.

# Finding the rhombohedral basis

Now we need to find the 4 different rhombohedral basis cases and code up their HNFs. We should also check to see how these basis compare to our original choices when it comes to efficiency.

In [11]:
A = np.transpose([[1,2,2],[2,1,2],[4,3,3]])
niggli_id(A)

('rhombohedral', 9, 3)

In [1]:
from opf_python.trig import rhom_2_4
count = 0
for i in range(501):
    count += len(rhom_2_4(i))
count

867

In [2]:
import time
start = time.time()
rhom_2_4(100000)
end = time.time()
print(end-start)

0.56950712204


In [1]:
from opf_python.trig import rhom_9
count = 0
for i in range(501):
    count += len(rhom_9(i))
count

867

In [2]:
import time
start = time.time()
rhom_9(100000)
end = time.time()
print(end-start)

0.0615818500519


In [1]:
from opf_python.trig import rhom_24
import time
start = time.time()
count = 0
for i in range(501):
    if i%100==0:
        end = time.time()
        print(i,end-start)
        start = time.time()
    count += len(rhom_24(i))
count

(0, 0.00016999244689941406)
(100, 0.005535125732421875)
(200, 0.00962209701538086)
(300, 0.01246786117553711)
(400, 0.027656078338623047)
(500, 0.02443981170654297)


867

In [2]:
import time
start = time.time()
rhom_24(100000)
end = time.time()
print(end-start)

0.151079893112


# Transformation Approach

Since solving for the HNFs in the Niggli basis is slow, 20+ times slower than in our basis choices, I'm curious if we can do something in which we map our supercells to the niggli supercell then the users supercell without having to deal with scaling or rotations. In essenc is:

S_u = N_u C_oinv H C_o C_uinv 

A symmetry preserving super cell of the users parent cell, where N_u is the niggli cell from the users basis, C_o is the transformation form our basis to the niggli cell, H is the HNF, C_u is the transformation form the users basis to the niggli cell, and S_u is the supercell.

In [3]:
from opf_python.trig import trig_spHNFs,rhom_9
import numpy as np
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id

In [2]:
O = np.transpose([[1,2,2],[2,1,2],[4,3,3]])
B = reduced_cell(O)
No = B.niggli
Co = B.C

In [3]:
def find_transformations(Nu,No):
    """Finds the S and R matrices that transform Nu to No.
    
    Args:
        Nu (numpy array): The users niggli cell.
        No (numpy array): Our niggli cell.
        
    Returns:
        R (numpy array): The rotation matrix.
        S (numpy array): The scaling matrix.
        
    Raises:
        ValueError if the determinant of R is not +/-1.
    """
    
    S = np.array([[np.linalg.norm(Nu[:,0])/np.linalg.norm(No[:,0]),0,0],
                 [0,np.linalg.norm(Nu[:,1])/np.linalg.norm(No[:,1]),0],
                 [0,0,np.linalg.norm(Nu[:,2])/np.linalg.norm(No[:,2])]])
    R = np.dot(np.dot(No,S),np.linalg.inv(Nu))
    
    #if not np.allclose(abs(np.linalg.det(R)),1):
    #    raise ValueError("The transformation found an "
    #                     "incorrect rotation vector.")
    return S, R

In [11]:
a1 = np.array([1,2,2])
a2 = np.array([2,1,2])
a3 = np.array([2,2,1])

As = [[a1,a2,a3],[a2+a1,a2+a3,a1+a3],[-a1,a2+a1+a1,a3-a1],[a1+a2+a3,a1+a2,a2+a3],
        [a1+a1,a2+a2,a3+a3]]
As = [np.transpose(i) for i in As]
crystal_fam=3
f_count = 0
for U in As:
    for kpd in range(1,501):
        HNFs = trig_spHNFs(kpd)
        #HNFs = rhom_9(kpd)
        Bu = reduced_cell(U)
        Cu = Bu.C
        Nu = Bu.niggli
        #S, R = find_transformations(Nu,No)
        for H in HNFs:
            L = np.dot(np.dot(O,H),np.linalg.inv(O))
            F = np.dot(np.linalg.inv(No),np.dot(np.dot(L,O),Co))
            Su = np.dot(np.dot(Nu,F),np.linalg.inv(Cu))
            #Su = np.dot(np.dot(np.linalg.inv(R),
            #                   np.dot(np.dot(np.dot(O,H),Co),S)),
            #            np.linalg.inv(Cu))
            #Su = np.dot(np.dot(Nu,H),np.linalg.inv(Cu))
            lat_name, niggli_n, lat_fam = niggli_id(Su)#,eps_=1E-3)
            r = np.linalg.inv(np.transpose(U))
            g = np.linalg.inv(np.transpose(Su))
            temp = np.round(np.dot(np.linalg.inv(g),r),3)
            if lat_fam > crystal_fam or not np.allclose(temp%1,0):
                f_count+=1
                print("Failed lat_fam",lat_fam,"int?",temp%1,'nn',niggli_n)
                SB = reduced_cell(Su)
                print("Su",Su)
                print("Su niggli",SB.niggli)
print(f_count)

0


In [5]:
Su = np.array([[ 230.,  232.,  232.],
       [ 308.,  306.,  308.],
       [ 232.,  232.,  230.]])
SB = reduced_cell(Su)
print(SB.niggli)
print(niggli_id(Su))

[[   2.    2.  258.]
 [  -2.    0.  256.]
 [   0.   -2.  256.]]
('rhombohedral', 9, 3)


In [5]:
A = np.array([[  1.,  -1.,  37.],
       [  1.,   0.,  26.],
       [  0.,   1., -12.]])
B = np.array([[ -1.,  -1., -13.],
       [  1.,   0.,  26.],
       [  0.,   1., -12.]])
np.dot(A,np.linalg.inv(B))

array([[-1.,  0., -2.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

Clearly this didn't work. Apparently, even though we can map from the niggli super cell to the users supercell this more complex evaluation fails to preserve the commensurability of the grid and the cell.

# Finding the 2 Hexagonal Niggli cells

We'll follow the same procedure as above to find and test the Hexagonal Niggli Cells.

In [12]:
A = np.transpose([[1,0,0],[0.5,-0.8660254037844386,0],[0,0,2]])
niggli_id(A)

('hexagonal', 12, 2)

In [20]:
A = np.transpose([[1,0,0],[-0.5,0.8660254037844386,0],[0,0,-2]])
niggli_setting(A)

[1.0, 0.99999999999999989, 4.0, 0.0, 0.0, -0.5]

In [8]:
A = np.transpose([[0,0,-0.5],[1,0,0],[-0.5,0.8660254037844386,0]])
niggli_setting(A)

[0.25, 1.0, 0.99999999999999989, -0.5, 0.0, 0.0]

In [11]:
from opf_python.hx import hex_12
count = 0
for i in range(501):
    count += len(hex_12(i))
count

1047

In [12]:
import time
start = time.time()
hex_12(100000)
end = time.time()
print(end-start)

0.381227016449


In [1]:
from opf_python.hx import hex_22
count = 0
for i in range(501):
    count += len(hex_22(i))
count

1047

In [2]:
import time
start = time.time()
hex_22(100000)
end = time.time()
print(end-start)

0.17823600769


# Finding the Second Simple Tetragonal Niggli Cell

The first niggli cell (11) for simple tetragonal was one we were already using. Now we just need the second (21).

In [21]:
A = np.transpose([[0,0,0.5],[1,0,0],[0,1,0]])
niggli_setting(A)
niggli_id(A)

('simple tetragonal', 21, 4)

In [7]:
from opf_python.pyniggli import reduced_cell
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 0. ,  0. ,  0.5],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ]])

In [16]:
from opf_python.stet import stet_11
count = 0
for i in range(501):
    count += len(stet_11(i))
count

1760

In [4]:
import time
start = time.time()
stet_21(100000)
end = time.time()
print(end-start)

0.0382399559021


# Finding the 4 Body Centered Tetragonal Cells

In [3]:
import numpy as np
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id

In [5]:
A = np.transpose([[-1,1,2],[1,-1,2],[1,1,-2]])
niggli_id(A)

('body centered tetragonal', 15, 4)

## Number 15

In [6]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 2.,  0.,  0.],
       [ 0.,  2.,  0.],
       [-1., -1.,  2.]])

In [1]:
from opf_python.body_tet import body_tet_15
count = 0
for i in range(501):
    count += len(body_tet_15(i))
count

1162

In [2]:
import time
start = time.time()
body_tet_15(100000)
end = time.time()
print(end-start)

1.80545282364


## Niggli 7

In [28]:
A = np.transpose([[-1, 1, 2],[1, 1.60788, -1.55394],
                     [1.95095, -1.41625, 0.433603]])
niggli_id(A)

('body centered tetragonal', 7, 4)

In [29]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1.95095 ,  1.41625 , -0.433603],
       [ 1.      , -1.      , -2.      ],
       [ 1.95095 ,  1.19163 ,  0.879663]])

In [1]:
from opf_python.body_tet import body_tet_7
count = 0
for i in range(1,501):
    count += len(body_tet_7(i))
count

1162

In [2]:
import time
start = time.time()
body_tet_7(100000)
end = time.time()
print(end-start)

0.460477113724


## Niggli 6

In [4]:
A = np.transpose([[-1, 1, 2],[1, 1.60788, -1.55394],
                     [1.80278, -1.47253, 0.762655]])
niggli_id(A)

('body centered tetragonal', 6, 4)

In [5]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1.      ,  1.      ,  2.      ],
       [ 1.      ,  1.60788 , -1.55394 ],
       [ 1.80278 , -1.47253 ,  0.762655]])

In [1]:
from opf_python.body_tet import body_tet_6
count = 0
for i in range(1,501):
    count += len(body_tet_6(i))
count

1162

In [2]:
import time
start = time.time()
body_tet_6(100000)
end = time.time()
print(end-start)

1.17160606384


## Niggli 18

In [8]:
A = np.transpose([[0,0,2],[1,-2,1],
                  [-2,-1,1]])
niggli_id(A)

('body centered tetragonal', 18, 4)

In [9]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 0,  0,  2],
       [ 1, -2,  1],
       [-2, -1,  1]])

In [1]:
from opf_python.body_tet import body_tet_18
count = 0
for i in range(1,501):
    count += len(body_tet_18(i))
count

1162

In [2]:
import time
start = time.time()
body_tet_18(100000)
end = time.time()
print(end-start)

1.24094605446


# Body Centered Ortho

In [13]:
A = np.transpose([[1,0,0],[0,2,0],[0,0,3]])
niggli_id(A)

('simple orthorhombic', 32, 5)

In [4]:
import numpy as np
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id

## Niggli 19

In [5]:
A = np.transpose([[-0.5,1,1.5],[0.5,-1,1.5],[0.5,1,-1.5]])
niggli_id(A)

('body centered orthorhombic', 19, 5)

In [6]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1. ,  0. ,  0. ],
       [-0.5, -1. ,  1.5],
       [-0.5,  1. ,  1.5]])

In [3]:
from opf_python.body_ortho import body_ortho_19
count = 0
for i in range(1,501):
    count += len(body_ortho_19(i))
count

15972

In [2]:
import time
start = time.time()
body_ortho_19(100000)
end = time.time()
print(end-start)

0.218292951584


## Niggli 8

In [6]:
A = np.transpose([[1, 1, 2],[1.41144,0.0885622,-2],
                 [-1.99868,1.21232,-0.731822]])
niggli_id(A)

('body centered orthorhombic', 8, 5)

In [46]:
B = reduced_cell(A,eps=1E-4)
np.transpose(B.niggli)

array([[ 1.       ,  1.       ,  2.       ],
       [ 1.41144  ,  0.0885622, -2.       ],
       [-1.99868  ,  1.21232  , -0.731822 ]])

In [1]:
from opf_python.body_ortho import body_ortho_8
count = 0
for i in range(1,501):
    count += len(body_ortho_8(i))
count

15972

In [2]:
import time
start = time.time()
body_ortho_8(100000)
end = time.time()
print(end-start)

4.48952507973


Looking into alternative basis choices.

In [12]:
a1 = np.transpose([[-0.5,1,1.5],[0.5,1,1.5],[0.5,1,-1.5]])
b1 = np.transpose([[1,0,0],[0.5,1,1.5],[0,0,3]])
np.dot(np.linalg.inv(a1),b1)

array([[-1.,  0.,  0.],
       [ 1.,  1.,  1.],
       [ 0.,  0., -1.]])

In [44]:
temp = [[ 0.,  1.,  0.],[ 1.,  0.,  1.],[ 0.,  1., -1.]]
np.linalg.det(temp)

1.0

In [47]:
Ap = np.dot(A,temp)
niggli_id(Ap)
np.transpose(Ap)

array([[ 1.41144  ,  0.0885622, -2.       ],
       [-0.99868  ,  2.21232  ,  1.268178 ],
       [ 3.41012  , -1.1237578, -1.268178 ]])

## Niggli 42

In [5]:
A = np.transpose([[1, 1, 1],[1.61803,-0.61803,-1],
                 [-1.53633,1.36706,-1.33073]])
niggli_id(A)

('body centered orthorhombic', 42, 5)

In [5]:
B = reduced_cell(A,eps=1E-4)
np.transpose(B.niggli)

array([[ 1.     ,  1.     ,  1.     ],
       [ 1.61803, -0.61803, -1.     ],
       [-1.53633,  1.36706, -1.33073]])

In [1]:
from opf_python.body_ortho import body_ortho_42
count = 0
for i in range(1,501):
    count += len(body_ortho_42(i))
count

15972

In [2]:
import time
start = time.time()
body_ortho_42(100000)
end = time.time()
print(end-start)

0.819437026978


Looking for alternative basis set.

In [47]:
#temp =[[-1.,  0.,  0.],
#       [ 1.,  1.,  1.],
#       [ 0.,  0., -1.]]
temp = [[-1,0,0],[0,0,-1],[0,-1,0]]
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
np.linalg.det(temp)

1.0

In [48]:
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

('body centered orthorhombic', 42, 5)


array([[-1.53633,  1.36706, -1.33073],
       [ 1.     ,  1.     ,  1.     ],
       [ 1.61803, -0.61803, -1.     ]])

In [27]:
temp

array([[-1.,  0., -1.],
       [ 1.,  1.,  2.],
       [ 0.,  0., -1.]])

# Face centered Orthorhombic

## Niggli Cell 26

In [4]:
import numpy as np
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id

In [15]:
A = np.transpose([[0,1,1.5],[0.5,0,1.5],[0,0,3]])
niggli_id(A)

('face centered orthorhombic', 26, 5)

In [4]:
A = np.transpose([[0,1,1.5],[0.5,0,1.5],
                 [0.5,1,0]])
niggli_id(A)

('face centered orthorhombic', 26, 5)

In [5]:
B = reduced_cell(A,eps=1E-4)
np.transpose(B.niggli)

array([[ 1. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ],
       [ 0.5,  0. ,  1.5]])

In [1]:
from opf_python.face_ortho import face_ortho_26
count = 0
for i in range(1,501):
    if i%100==0:
        print(i)
    count += len(face_ortho_26(i))
count

100
200
300
400
500


15246

In [2]:
import time
start = time.time()
face_ortho_26(100000)
end = time.time()
print(end-start)

0.538698911667


## Niggli 16

In [6]:
A = np.transpose([[1,1,-1],[0.779796,-1.1798,1],
                 [-1.04442,-1.43973,-1.68415]])
niggli_id(A)

('face centered orthorhombic', 16, 5)

In [21]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  1.      , -1.      ],
       [ 0.779796, -1.1798  ,  1.      ],
       [-1.04442 , -1.43973 , -1.68415 ]])

In [3]:
from opf_python.face_ortho import face_ortho_16
count = 0
for i in range(1,501):
    count += len(face_ortho_16(i))
count

15246

In [4]:
import time
start = time.time()
face_ortho_16(100000)
end = time.time()
print(end-start)

2.31728506088


Looking for alternative basis.

In [5]:
temp = [[-1,0,0],[0,0,-1],[0,-1,0]]
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
np.linalg.det(temp)

1.0

In [9]:
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

12
('face centered orthorhombic', 16, 5)


array([[-1.04442 , -1.43973 , -1.68415 ],
       [ 1.      ,  1.      , -1.      ],
       [ 0.779796, -1.1798  ,  1.      ]])

In [133]:
Ap = np.dot(A,[[1,-1,1],[0,-1,1],[0,0,1]])
np.transpose(Ap)
#niggli_id(Ap)

array([[ 1.      ,  1.      , -1.      ],
       [-1.779796,  0.1798  ,  0.      ],
       [ 0.735376, -1.61953 , -1.68415 ]])

# Base Centered Orthorhombic

## Niggli Cell 38

In [1]:
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id
import numpy as np

In [2]:
A = np.transpose([[0.5,1,0],[0.5,-1,0],[0,0,3]])
niggli_id(A)

('base centered orthorhombic', 38, 5)

In [21]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1. ,  0. ,  0. ],
       [ 0.5,  1. ,  0. ],
       [ 0. ,  0. ,  3. ]])

## Niggli Cell 13

In [32]:
A = np.transpose([[1,1,1],
                  [1,-1,-1],
                  [0,-1.73205,1.73205]])
niggli_id(A)

('base centered orthorhombic', 13, 5)

In [34]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.     ,  1.     ,  1.     ],
       [ 1.     , -1.     , -1.     ],
       [ 0.     , -1.73205,  1.73205]])

## Niggli Cell 23

In [7]:
A = np.transpose([[1,1,1],
                 [2,-1,-1],
                 [-0.3333333,-1.54116,1.87449]])
niggli_id(A)

4


('base centered orthorhombic', 23, 5)

In [38]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.       ,  1.       ,  1.       ],
       [ 2.       , -1.       , -1.       ],
       [-0.3333333, -1.54116  ,  1.87449  ]])

In [1]:
from opf_python.base_ortho import base_ortho_23
count = 0
for i in range(1,501):
    count += len(base_ortho_23(i))
count

19534

In [2]:
import time
start = time.time()
base_ortho_23(100000)
end = time.time()
print(end-start)

1.24802517891


Looking for better basis.

In [8]:
temp = [[-1,0,0],[0,0,-1],[0,-1,0]]
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
np.linalg.det(temp)

1.0

In [14]:
A = np.transpose([[1,1,1],[2,-1,-1],[-0.33333333,-1.54116,1.87449]])
niggli_id(A)

('base centered orthorhombic', 23, 5)

In [9]:
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

14
('base centered orthorhombic', 23, 5)


array([[-0.3333333, -1.54116  ,  1.87449  ],
       [ 1.       ,  1.       ,  1.       ],
       [ 2.       , -1.       , -1.       ]])

## Niggli Cell 40

In [40]:
A = np.transpose([[1,1,1],
                 [1.61803,-0.618034,-1],
                 [-1.05557,1.99895,-0.943376]])
niggli_id(A)

('base centered orthorhombic', 40, 5)

In [41]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  1.      ,  1.      ],
       [ 1.61803 , -0.618034, -1.      ],
       [-1.05557 ,  1.99895 , -0.943376]])

In [1]:
from opf_python.base_ortho import base_ortho_40
count = 0
for i in range(1,501):
    count += len(base_ortho_40(i))
count

19534

In [2]:
import time
start = time.time()
base_ortho_40(100000)
end = time.time()
print(end-start)

1.58012008667


## Niggli Cell 36

In [25]:
A = np.transpose([[1, 1, 1],
                  [1.41421, -1.41421, 0],
                  [-1.43541, -1.43541, 1.37083]])
niggli_id(A)

('base centered orthorhombic', 36, 5)

In [27]:
from opf_python.base_ortho import base_ortho_36
count = 0
for i in range(1,501):
    count += len(base_ortho_36(i))
count

19534

In [28]:
import time
start = time.time()
base_ortho_36(100000)
end = time.time()
print(end-start)

1.27605319023


# Simple Monoclinic

## Niggli Cell 33

In [4]:
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id
import numpy as np

In [44]:
A = np.transpose([[2,0,0],
                  [0,2,0],
                  [0.5,0,2]])
niggli_id(A)

('simple monoclinic', 33, 6)

In [45]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 2. ,  0. ,  0. ],
       [ 0. , -2. ,  0. ],
       [-0.5,  0. , -2. ]])

## Niggli Cell 35

In [48]:
A = np.transpose([[1,1,1],
                  [1.61803,-0.618034,-1],
                  [-0.668912,1.96676,-1.29785]])
niggli_id(A)

('simple monoclinic', 35, 6)

In [49]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  1.      ,  1.      ],
       [ 1.61803 , -0.618034, -1.      ],
       [-0.668912,  1.96676 , -1.29785 ]])

## Niggli Cell 34

In [57]:
A = np.transpose([[1,1,1],
                  [1.22474487,-1.22474487,-1],
                  [-0.165985087,-1.64308297,1.80906806]])
niggli_id(A)

('simple monoclinic', 34, 6)

In [61]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.        ,  1.        ,  1.        ],
       [ 1.22474487, -1.22474487, -1.        ],
       [-0.16598509, -1.64308297,  1.80906806]])

# Base Centered Monoclinic

## Niggli Basis 14

In [3]:
import numpy as np
from opf_python.pyniggli import reduced_cell
from opf_python.niggli_lat_id import niggli_id

In [63]:
A = np.transpose([[1,1,0],
                  [1,-1,0],
                  [0.5,0,2]])
niggli_id(A)

('base centered monoclinic', 14, 6)

In [64]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1. , -1. ,  0. ],
       [-1. ,  1. ,  0. ],
       [ 0.5,  0. ,  2. ]])

## Niggli Cell 10

In [7]:
A = np.transpose([[1,1,1],
                  [1,-1,1],
                  [-1.46391,0,1.96391]])
niggli_id(A)

('bace centered monoclinic', 10, 6)

In [70]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.     ,  1.     ,  1.     ],
       [ 1.     , -1.     ,  1.     ],
       [-1.46391,  0.     ,  1.96391]])

Looking for alternative basis.

In [9]:
#temp = [[1,0,0],[0,1,0],[0,0,1]]
temp = [[1,1,0],[0,-1,0],[0,0,1]]
temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

-1.0
('bace centered monoclinic', 10, 6)


array([[-1.46391,  0.     ,  1.96391],
       [ 1.     ,  1.     ,  1.     ],
       [ 0.     ,  2.     ,  0.     ]])

HNFs match case 39.

## Niggli Basis 17

In [10]:
A = np.transpose([[1,1,1],
                  [0.809568,-1.15957,-1],
                  [-1.05387,-1.61088,1.51474]])
niggli_id(A)

('bace centered monoclinic', 17, 6)

In [72]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  1.      ,  1.      ],
       [ 0.809568, -1.15957 , -1.      ],
       [-1.05387 , -1.61088 ,  1.51474 ]])

Looking for alternative basis.

In [26]:
#temp = [[1,0,0],[0,1,0],[0,0,1]]
temp = [[1,0,1],[0,-1,0],[0,0,1]]
temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('bace centered monoclinic', 17, 6)


array([[-0.05387 , -0.61088 ,  2.51474 ],
       [ 1.      ,  1.      ,  1.      ],
       [ 1.809568, -0.15957 ,  0.      ]])

Now matches basis 39.

## Niggli Basis 20

In [27]:
A = np.transpose([[1,1,1],
                  [1.70119,-1.45119,1],
                  [-1.0034,0.0189395,2.23446]])
niggli_id(A)

('base centered monoclinic', 20, 6)

In [74]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.       ,  1.       ,  1.       ],
       [ 1.70119  , -1.45119  ,  1.       ],
       [-1.0034   ,  0.0189395,  2.23446  ]])

Looking for alternative basis.

In [35]:
temp = [[1,0,0],[0,1,0],[0,0,1]]
#temp = [[1,0,1],[0,-1,0],[0,0,1]]
temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
#temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('base centered monoclinic', 20, 6)


array([[ 1.       ,  1.       ,  1.       ],
       [ 1.70119  , -1.45119  ,  1.       ],
       [ 0.69779  , -1.4322505,  3.23446  ]])

In [1]:
from opf_python.base_mono import base_mono_20
count = 0
for i in range(1,201):
    count += len(base_mono_20(i))
count

53842

In [2]:
import time
start = time.time()
base_mono_20(100000)
end = time.time()
print(end-start)

2.81764507294


## Niggli Cell 25

In [2]:
A = np.transpose([[1,1,1],
                  [1.45119,-1.70119,-1],
                  [-1.16241,-1.56776,1.48018]])
niggli_id(A)

('base centered monoclinic', 25, 6)

In [3]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.     ,  1.     ,  1.     ],
       [ 1.45119, -1.70119, -1.     ],
       [-1.16241, -1.56776,  1.48018]])

Looking for alternative basis.

In [4]:
import numpy as np
temp = [[1,0,0],[0,1,0],[0,0,1]]
#temp = [[1,0,1],[0,-1,0],[0,0,1]]
temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
#temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('base centered monoclinic', 25, 6)


array([[ 1.     ,  1.     ,  1.     ],
       [ 1.45119, -1.70119, -1.     ],
       [ 0.28878, -3.26895,  0.48018]])

Same as cell 20

## Niggli Cell 27

In [7]:
A = np.transpose([[1,1,0],
                  [1.618033,-0.618033,1],
                  [-0.464824,1.464824,1.907413]])
niggli_id(A)

('base centered monoclinic', 27, 6)

In [98]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[-1.      , -1.      ,  0.      ],
       [-1.618033,  0.618033, -1.      ],
       [-1.464824,  0.464824,  1.907413]])

Looking for alternative basis.

In [9]:
import numpy as np
temp = [[1,0,0],[0,1,0],[0,0,1]]
#temp = [[1,0,1],[0,-1,0],[0,0,1]]
#temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('base centered monoclinic', 27, 6)


array([[ 0.464824, -1.464824, -1.907413],
       [-1.618033,  0.618033, -1.      ],
       [-1.      , -1.      ,  0.      ]])

In [1]:
from opf_python.base_mono import base_mono_27
count = 0
for i in range(1,201):
    count += len(base_mono_27(i))
count

53842

In [2]:
import time
start = time.time()
base_mono_27(100000)
end = time.time()
print(end-start)

4.70454502106


## Niggli Basis 28

In [8]:
A = np.transpose([[1,1,0],
                  [1.44896,-0.948958,1],
                  [-0.342424,1.342424,2.02006]])
niggli_id(A)

('base centered monoclinic', 28, 6)

In [100]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  1.      ,  0.      ],
       [ 1.44896 , -0.948958,  1.      ],
       [-0.342424,  1.342424,  2.02006 ]])

In [1]:
from opf_python.base_mono import base_mono_28
count = 0
for i in range(1,201):
    count += len(base_mono_28(i))
count

53842

In [2]:
import time
start = time.time()
base_mono_28(100000)
end = time.time()
print(end-start)

4.18533301353


Looking for alternative basis.

In [9]:
import numpy as np
temp = [[1,0,0],[0,1,0],[0,0,1]]
#temp = [[1,0,1],[0,-1,0],[0,0,1]]
#temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
#temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('base centered monoclinic', 28, 6)


array([[-1.44896 ,  0.948958, -1.      ],
       [-1.      , -1.      ,  0.      ],
       [ 0.342424, -1.342424, -2.02006 ]])

## Niggli Cell 29

In [5]:
A = np.transpose([[1,1,0],
                  [1.61803,-0.618034,1],
                  [-0.666125,1.16613,2.04852]])
niggli_id(A)

('base centered monoclinic', 29, 6)

In [102]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  1.      ,  0.      ],
       [ 1.61803 , -0.618034,  1.      ],
       [-0.666125,  1.16613 ,  2.04852 ]])

Looking for alternative basis.

In [6]:
import numpy as np
temp = [[1,0,0],[0,1,0],[0,0,1]]
#temp = [[1,0,1],[0,-1,0],[0,0,1]]
#temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('base centered monoclinic', 29, 6)


array([[-0.666125,  1.16613 ,  2.04852 ],
       [ 1.      ,  1.      ,  0.      ],
       [ 1.61803 , -0.618034,  1.      ]])

In [1]:
from opf_python.base_mono import base_mono_29
count = 0
for i in range(1,201):
    count += len(base_mono_29(i))
count

53842

In [2]:
import time
start = time.time()
base_mono_29(100000)
end = time.time()
print(end-start)

3.89106893539


## Niggli Cell 30

In [105]:
A = np.transpose([[1,1,0],
                  [1.61803,-0.618034,1],
                  [-0.0361373,0.536137,2.38982]])
niggli_id(A)

('base centered monoclinic', 30, 6)

In [106]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.       ,  1.       ,  0.       ],
       [ 1.61803  , -0.618034 ,  1.       ],
       [-0.0361373,  0.536137 ,  2.38982  ]])

Same as basis 29

## Niggli Cell 41

In [4]:
A = np.transpose([[1,0,1],
                  [1,-1.41421,-1],
                  [-1.85397,-0.854143,1.35397]])
niggli_id(A)

('base centered monoclinic', 41, 6)

In [110]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  0.      ,  1.      ],
       [ 1.      , -1.41421 , -1.      ],
       [-1.85397 , -0.854143,  1.35397 ]])

Looking for alternative basis.

In [5]:
import numpy as np
temp = [[1,0,0],[0,1,0],[0,0,1]]
#temp = [[1,0,1],[0,-1,0],[0,0,1]]
#temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('base centered monoclinic', 41, 6)


array([[-1.      ,  0.      , -1.      ],
       [ 1.85397 ,  0.854143, -1.35397 ],
       [-1.      ,  1.41421 ,  1.      ]])

In [1]:
from opf_python.base_mono import base_mono_41
count = 0
for i in range(1,201):
    count += len(base_mono_41(i))
count

53842

In [2]:
import time
start = time.time()
base_mono_41(100000)
end = time.time()
print(end-start)

4.30097007751


## Niggli Cell 37

In [4]:
A = np.transpose([[1,0,1],
                  [1,-1.41421,-1],
                  [-1.79092,-1.47209,0.790922]])
niggli_id(A)

('base centered monoclinic', 37, 6)

In [114]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  0.      ,  1.      ],
       [ 1.      , -1.41421 , -1.      ],
       [-1.79092 , -1.47209 ,  0.790922]])

Looking for alternative basis.

In [5]:
import numpy as np
temp = [[1,0,0],[0,1,0],[0,0,1]]
#temp = [[1,0,1],[0,-1,0],[0,0,1]]
#temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('base centered monoclinic', 37, 6)


array([[-1.79092 , -1.47209 ,  0.790922],
       [ 1.      ,  0.      ,  1.      ],
       [ 1.      , -1.41421 , -1.      ]])

In [1]:
from opf_python.base_mono import base_mono_37
count = 0
for i in range(1,201):
    count += len(base_mono_37(i))
count

53842

In [2]:
import time
start = time.time()
base_mono_37(100000)
end = time.time()
print(end-start)

2.95140385628


## Niggli Cell 39

In [5]:
A = np.transpose([[1,0,1],
                  [0,-1.73205,-1],
                  [-1.66542,-0.672857,1.66542]])
niggli_id(A)

('base centered monoclinic', 39, 6)

In [117]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.      ,  0.      ,  1.      ],
       [ 0.      , -1.73205 , -1.      ],
       [-1.66542 , -0.672857,  1.66542 ]])

Looking for alternative basis.

In [6]:
import numpy as np
temp = [[1,0,0],[0,1,0],[0,0,1]]
#temp = [[1,0,1],[0,-1,0],[0,0,1]]
#temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
#temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('base centered monoclinic', 39, 6)


array([[ 0.      ,  1.73205 ,  1.      ],
       [-1.      ,  0.      , -1.      ],
       [ 1.66542 ,  0.672857, -1.66542 ]])

## Niggli Cell 43

In [5]:
A = np.transpose([[1,1,1],
                  [1.64194,-1.14194,-1],
                  [-1.39716,-1.34718,1.49434]])
niggli_id(A)

('body centered monoclinc', 43, 6)

In [124]:
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 1.     ,  1.     ,  1.     ],
       [ 1.64194, -1.14194, -1.     ],
       [-1.39716, -1.34718,  1.49434]])

Looking for alternative basis.

In [9]:
#temp = [[1,0,0],[0,1,0],[0,0,1]]
temp = [[1,1,0],[0,1,0],[1,0,1]]
#temp = np.dot(temp,[[1,0,0],[0,1,1],[0,0,1]])
#temp = np.dot(temp,[[1,1,0],[0,-1,0],[0,0,1]])
#temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
#temp = np.dot(temp,[[-1,0,0],[0,0,-1],[0,-1,0]])
#temp = np.dot(temp,[[0,-1,0],[-1,0,0],[0,0,-1]])
print(np.linalg.det(temp))
Ap = np.dot(A,temp)
print(niggli_id(Ap))
np.transpose(Ap)

1.0
('body centered monoclinc', 43, 6)


array([[-0.39716, -0.34718,  2.49434],
       [ 2.64194, -0.14194,  0.     ],
       [-1.39716, -1.34718,  1.49434]])

In [1]:
from opf_python.base_mono import base_mono_43
count = 0
for i in range(1,201):
    count += len(base_mono_43(i))
count

53842

In [2]:
import time
start = time.time()
base_mono_43(100000)
end = time.time()
print(end-start)

4.56314086914


In [23]:
niggli_id(A,G=[1,2,3,-.1,-.2,-.3])

('triclinic', 44, 7)